In [2]:
%%sql
CREATE DATABASE IF NOT EXISTS dev_temp_curated;

++
||
++
++

In [7]:
%%sql
drop table dev_temp_raw._temp_table

++
||
++
++

In [9]:
%%sql
CREATE TABLE IF NOT EXISTS dev_temp_raw._temp_table (
    id                    integer,
    name                  string,
    age                   integer,
    created_at            timestamp
)
USING iceberg
PARTITIONED BY (days(created_at))

++
||
++
++

In [3]:
from datetime import datetime

schema = spark.table("climate.weather").schema

data = [
    (datetime(2023,8,16), 76.2, 40.951908, -74.075272, "Partially sunny", 0.0, 3.5),
    (datetime(2023,8,17), 82.5, 40.951908, -74.075272, "Sunny", 0.0, 1.2),
    (datetime(2023,8,18), 70.9, 40.951908, -74.075272, "Cloudy", .5, 5.2)
  ]

df = spark.createDataFrame(data, schema)
df.writeTo("climate.weather").append()

In [4]:
df.show()

+-------------------+----+---------+----------+---------------+------+----------+
|           datetime|temp|      lat|      long| cloud_coverage|precip|wind_speed|
+-------------------+----+---------+----------+---------------+------+----------+
|2023-08-16 00:00:00|76.2|40.951908|-74.075272|Partially sunny|   0.0|       3.5|
|2023-08-17 00:00:00|82.5|40.951908|-74.075272|          Sunny|   0.0|       1.2|
|2023-08-18 00:00:00|70.9|40.951908|-74.075272|         Cloudy|   0.5|       5.2|
+-------------------+----+---------+----------+---------------+------+----------+



In [8]:
%%sql
select
    month(datetime), avg(wind_speed) 
from climate.weather
where lower(cloud_coverage) like '%sun%'
group by month(datetime)

month(datetime),avg(wind_speed)
8,2.35


### Test Postgres
---

In [6]:
tables = ["customer_acquisition_channels", "customers", "inventory", "order_items", "orders", "product_categories", "products"]
tables = ["customers"]
jdbc_url = "jdbc:postgresql://postgres/ecommerce"
jdbc_properties = {
    "user": "postgres",
    "password": "postgres",
    "driver": "org.postgresql.Driver"
}
def get_df(jdbc_url, jdbc_prop, table):
    # Read data from PostgreSQL
    print(f"Reading data from PostgreSQL, table: {table}...")
    query = f"""
        select * from {table}
        where registration_date > '2020-01-05 02:23:48.000'
    """
    # postgres_df = spark.read.jdbc(url=jdbc_url, query=query, table=table, properties=jdbc_prop)
    postgres_df = spark.read.format('jdbc') \
                    .option("url", jdbc_url) \
                    .option("query",query) \
                    .option("user","postgres") \
                    .option("password","postgres") \
                    .option("driver","org.postgresql.Driver") \
                    .load()
    print(postgres_df.show())

for table in tables:
    get_df(jdbc_url, jdbc_properties, table)

Reading data from PostgreSQL, table: customers...
+-----------+---------+------+-------------+--------------+-------+-------------------+----------------------+
|customer_id|     name|gender|        email|         phone|country|  registration_date|acquisition_channel_id|
+-----------+---------+------+-------------+--------------+-------+-------------------+----------------------+
|  999999991|test user|     M|test@mail.com|+1232131231231|    ABC|2020-01-05 02:23:49|                     3|
+-----------+---------+------+-------------+--------------+-------+-------------------+----------------------+

None


In [4]:
# Write data to MinIO in Iceberg format
iceberg_table = "demo.ecommerce.customers"

print(f"Writing data to Iceberg table: {iceberg_table}")
postgres_df.writeTo(iceberg_table).createOrReplace()

print("Data migration completed successfully!")

Writing data to Iceberg table: demo.ecommerce.customers


Data migration completed successfully!


In [11]:
%%sql
-- select * from ecommerce.customers
select * from dev_temp_raw._temp_table

id,name,age,created_at
1,agung,120,2024-12-13 15:58:07.574431
2,tri,120,2024-12-13 15:58:07.574431
3,atd,1,2024-12-13 15:58:07.574431
1,agung,120,2024-12-13 15:58:07.574431
2,tri,120,2024-12-13 15:58:07.574431
3,atd,1,2024-12-13 15:58:07.574431
1,agung,120,2024-12-13 15:58:07.574431
2,tri,120,2024-12-13 15:58:07.574431
3,atd,1,2024-12-13 15:58:07.574431


In [6]:
spark.table("ecommerce.customers").schema

StructType([StructField('customer_id', IntegerType(), True), StructField('name', StringType(), True), StructField('gender', StringType(), True), StructField('email', StringType(), True), StructField('phone', StringType(), True), StructField('country', StringType(), True), StructField('registration_date', TimestampType(), True), StructField('acquisition_channel_id', IntegerType(), True)])

In [11]:
query = '''select max(registration_date) from customers'''
df = spark.read.format('jdbc') \
        .option("url", "jdbc:postgresql://postgres/ecommerce") \
        .option("query",query) \
        .option("user","postgres") \
        .option("password","postgres") \
        .option("driver","org.postgresql.Driver") \
        .load()
last_date = df.first()[0]
last_date

datetime.datetime(2020, 1, 5, 2, 23, 49)

In [12]:
query = f'''select max(registration_date) from customers where registration_date < '{last_date}' '''
df = spark.read.format('jdbc') \
        .option("url", "jdbc:postgresql://postgres/ecommerce") \
        .option("query",query) \
        .option("user","postgres") \
        .option("password","postgres") \
        .option("driver","org.postgresql.Driver") \
        .load()
df.show()

+-------------------+
|                max|
+-------------------+
|2020-01-05 02:23:48|
+-------------------+



'2020-01-05 02:23:49'

In [1]:
spark